Importing Libraries

In [16]:
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

Reading Excel

In [17]:
df = pd.read_excel("train.xlsx")
df

,Unnamed: 0.1,Unnamed: 0,Role,User Story,Verbs,Adjectives,Nouns,Imp Words
0,0,0,Data user,I want to have the 12-19-2017 deletions proce...,['processed'],['12-19-2017'],['deletions'],"['want', '12-19-2017', 'deletions', 'processed']"
1,1,1,UI designer,"I want to redesign the Resources page, so tha...",[],[],"['redesign', 'Resources', 'matches', 'Broker',...","['want', 'redesign', 'Resources', 'matches', '..."
2,2,2,UI designer,I want to report to the Agencies about user t...,['making'],"['aware', 'better']","['report', 'Agencies', 'user', 'testing', 'con...","['want', 'report', 'Agencies', 'user', 'testin..."
3,3,3,UI designer,I want to move on to round 2 of DABS or FABS ...,['landing'],[],"['move', 'DABS', 'FABS', 'edits', 'approvals',...","['want', 'move', 'DABS', 'FABS', 'landing', 'e..."
4,4,4,UI designer,I want to move on to round 2 of Homepage edit...,[],[],"['move', 'edits', 'approvals', 'leadership']","['want', 'move', 'edits', 'get', 'approvals', ..."
...,...,...,...,...,...,...,...,...
1661,1661,1661,admin,I want to know when Zoonibot should give an e...,[],[],"['Zoonibot', 'explanation']","['want', 'know', 'Zoonibot', 'explanation']"
1662,1662,1662,admin,I want to know what Zoonibot should say to a ...,[],[],"['Zoonibot', 'volunteer']","['want', 'know', 'Zoonibot', 'volunteer']"
1663,1663,1663,admin,I want to group subjects by similarity.,['subjects'],[],"['group', 'similarity']","['want', 'group', 'subjects', 'similarity']"
1664,1664,1664,admin,I want to recommend different projects to vol...,['based'],"['recommend', 'previous']","['projects', 'volunteers', 'experiences']","['want', 'recommend', 'projects', 'volunteers'..."


Reading a Column User story from dataframe

In [18]:
sentences = df['User Story']
sentences

0        I want to have the 12-19-2017 deletions proce...
1        I want to redesign the Resources page, so tha...
2        I want to report to the Agencies about user t...
3        I want to move on to round 2 of DABS or FABS ...
4        I want to move on to round 2 of Homepage edit...
                              ...                        
1661     I want to know when Zoonibot should give an e...
1662     I want to know what Zoonibot should say to a ...
1663              I want to group subjects by similarity.
1664     I want to recommend different projects to vol...
1665     I want to see a summary of articles, so that ...
Name: User Story, Length: 1666, dtype: object

Load the custom stop words

In [19]:
def load_stop(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        stopwords_list = file.read().splitlines()
    return stopwords_list

Tokenize and Split to verbs, adjectives,noun

In [20]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag

def load_stopwords(file_path):
    with open(file_path, 'r') as file:
        stopwords_list = file.read().splitlines()
    return stopwords_list

def custom_tokenizer(text):
    stopwords_list = load_stop("./stopwords-en.txt")
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    filtered_tokens = [token for token in tokens if token.lower() not in stopwords_list]
    return filtered_tokens

def identify_pos(tokens):
    # Tag tokens with parts of speech
    pos_tags = pos_tag(tokens)
    
    # verbs = [word for word, tag in pos_tags if tag.startswith('VB')]
    # adjectives = [word for word, tag in pos_tags if tag.startswith('JJ')]
    # nouns = [word for word, tag in pos_tags if tag.startswith('NN')]
    # pos_list = verbs + adjectives + nouns
    pos_list = []
    
    for word, tag in pos_tags:
        if tag.startswith('VB') or tag.startswith('JJ') or tag.startswith('NN'):
            pos_list.append(word)
    return pos_list


Processing each Row in Dataframe

In [21]:
def process_row(row):
    text = row['User Story']
    filtered_tokens = custom_tokenizer(text)
    words = identify_pos(filtered_tokens)
    row['Imp Words'] = words
    return row

In [33]:
# df = df.apply(process_row,axis = 1)
df.drop(["Unnamed:0","Unnamed: 0.1","Verbs","Adjectives","Nouns","Role"],axis=1)

KeyError: "['Unnamed:0'] not found in axis"

In [58]:
df.to_excel("dfidf.xlsx")

Test Data

In [59]:
# txt = "I want to know when Zoonibot should comment on a subject."
# # txt
# t = custom_tokenizer(txt)
# l = identify_pos(t)

In [8]:
df['Imp Words'][0][0]

'want'

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Step 1: Tokenize the text in each row
df['Tokens'] = df['Imp Words']

# Step 2: Convert the tokenized text into strings
df['Text_str'] = df['Tokens'].apply(lambda x: ' '.join(x))

# Step 3: Compute TF-IDF scores for each word in the corpus
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Text_str'])

# Step 4: Extract TF-IDF scores for each word
feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

# Step 5: Assign TF-IDF scores to each word in the dataframe
df = pd.concat([df,tfidf_df],axis=1)

# Display the updated dataframe
print(df)


      Unnamed: 0.1  Unnamed: 0         Role  \
0                0           0    Data user   
1                1           1  UI designer   
2                2           2  UI designer   
3                3           3  UI designer   
4                4           4  UI designer   
...            ...         ...          ...   
1661          1661        1661        admin   
1662          1662        1662        admin   
1663          1663        1663        admin   
1664          1664        1664        admin   
1665          1665        1665        admin   

                                             User Story          Verbs  \
0      I want to have the 12-19-2017 deletions proce...  ['processed']   
1      I want to redesign the Resources page, so tha...             []   
2      I want to report to the Agencies about user t...     ['making']   
3      I want to move on to round 2 of DABS or FABS ...    ['landing']   
4      I want to move on to round 2 of Homepage edit...          

In [10]:
df.to_excel("test.xlsx")

In [14]:
df.rows.size

1666

In [15]:
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Sample dataframe with existing user stories and TF-IDF scores
# Replace this with your actual dataframe containing TF-IDF scores
# Make sure it includes TF-IDF scores for each word
data = {
    'Text': ['The quick brown fox jumps over the lazy dog.',
             'The sun is shining brightly today.',
             # Add more user stories as needed
            ],
    # Example TF-IDF scores for each word in the existing user stories
    'want_tfidf': [0.0, 0.0],
    'quick_tfidf': [0.0, 0.3],
    'brown_tfidf': [0.4, 0.0],
    # Add more TF-IDF columns as needed
}
df = pd.DataFrame(data)

# New user story
new_user_story = "A lazy cat is sleeping on the couch."

# Step 1: Tokenize the new user story
new_tokens = word_tokenize(new_user_story)

# Step 2: Convert the tokenized new user story into a string
new_text_str = ' '.join(new_tokens)

# Step 3: Compute TF-IDF scores for the new user story
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform([new_text_str] + df['Text'])

# Step 4: Calculate the cosine similarity between the TF-IDF vectors
cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])[0]

# Display the cosine similarities
print("Cosine Similarities with Existing User Stories:")
for i, similarity in enumerate(cosine_similarities):
    print(f"User Story {i+1}: {similarity}")


Cosine Similarities with Existing User Stories:
User Story 1: 0.5604726472929079
